# 特定の日時のリストを取得する 

In [4]:
%cd ..

E:\システムトレード入門\trade_system_git_workspace


In [37]:
import jpholiday
import workdays
from pytz import timezone, utc
import datetime

In [38]:
import numpy as np
import pandas as pd

In [39]:
def check_jst_datetimes_to_naive(*arg_datetimes):
    """
    ＊*今のところ，ローカルが東京でないnaiveなdatetimeはそのまま通してしまう
    引数のタイムゾーンが同じかどうかチェックし，存在するなら日本であるかチェック
    awareな場合は，naiveに変更
    """
    jst_timezone = timezone("Asia/Tokyo")
    tz_info_set = set([one_datetime.tzinfo for one_datetime in arg_datetimes])
    if len(tz_info_set) > 1:
        raise Exception("timezones are different")
        
    datetimes_tzinfo = list(tz_info_set)[0]
    
    if datetimes_tzinfo is not None:  # 長さが1のはず
        if timezone(str(datetimes_tzinfo)) != jst_timezone:
            raise Exception("timezones must be Asia/Tokyo")
        # naiveなdatetimeに変更
        arg_datetimes = [one_datetime.replace(tzinfo=None) for one_datetime in arg_datetimes]
        
    return tuple(arg_datetimes)

## 営業日・休日を取得する 

### 祝日のndarrayを取得する

以下が素直な実装だが，遅い．そこで一度しか呼ばないようにすべき

In [40]:
def get_holiday_jp(start_datetime, end_datetime, with_name=False, to_date=True):
    """
    期間を指定して祝日を取得
    
    start_datetime: datetime.datetime
        開始時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    end_datetime: datetime.datetime
        終了時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    eith_name: bool
        休日の名前を出力するかどうか
    to_date: bool
        出力をdatetime.datetimeにするかdatetime.dateにするか
    """
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)
    
    if to_date:
        start_datetime = start_datetime.date()
        end_datetime = end_datetime.date()
    
    holydays_array = np.array(jpholiday.between(start_datetime, end_datetime))
    
    if not with_name:
        return holydays_array[:,0].copy()
    
    return holydays_array

In [41]:
start_datetime = datetime.datetime(2019, 1, 1, 0, 0, 0)
end_datetime = datetime.datetime(2020, 12, 31, 0, 0, 0)

#jst_timezone = timezone("Asia/Tokyo")
#start_datetime = jst_timezone.localize(datetime.datetime(2019, 1, 1, 0, 0, 0))
#end_datetime = jst_timezone.localize(datetime.datetime(2020, 12, 31, 0, 0, 0))

holydays = get_holiday_jp(start_datetime, end_datetime, with_name=False, to_date=True)

In [42]:
holydays

array([datetime.date(2019, 1, 1), datetime.date(2019, 1, 14),
       datetime.date(2019, 2, 11), datetime.date(2019, 3, 21),
       datetime.date(2019, 4, 29), datetime.date(2019, 4, 30),
       datetime.date(2019, 5, 1), datetime.date(2019, 5, 2),
       datetime.date(2019, 5, 3), datetime.date(2019, 5, 4),
       datetime.date(2019, 5, 5), datetime.date(2019, 5, 6),
       datetime.date(2019, 7, 15), datetime.date(2019, 8, 11),
       datetime.date(2019, 8, 12), datetime.date(2019, 9, 16),
       datetime.date(2019, 9, 23), datetime.date(2019, 10, 14),
       datetime.date(2019, 10, 22), datetime.date(2019, 11, 3),
       datetime.date(2019, 11, 4), datetime.date(2019, 11, 23),
       datetime.date(2020, 1, 1), datetime.date(2020, 1, 13),
       datetime.date(2020, 2, 11), datetime.date(2020, 2, 23),
       datetime.date(2020, 2, 24), datetime.date(2020, 3, 20),
       datetime.date(2020, 4, 29), datetime.date(2020, 5, 3),
       datetime.date(2020, 5, 4), datetime.date(2020, 5, 5),


### 営業日を取得

内部で外部の関数等を利用するので，jitコンパイルはできない．そこで，あらかじめ数年分のholidayを得て置き，それと照らし合わせることにする．

In [43]:
holiday_start_year = 2015
holiday_end_year = 2020

holidays_date_array = get_holiday_jp(start_datetime=datetime.datetime(holiday_start_year,1,1,0,0,0),
                                end_datetime=datetime.datetime(holiday_end_year,12,31,23,59,59),
                                with_name=False,
                               )
print(holidays_date_array)

holidays_datetimeindex = pd.DatetimeIndex(holidays_date_array)
print(holidays_datetimeindex)

[datetime.date(2015, 1, 1) datetime.date(2015, 1, 12)
 datetime.date(2015, 2, 11) datetime.date(2015, 3, 21)
 datetime.date(2015, 4, 29) datetime.date(2015, 5, 3)
 datetime.date(2015, 5, 4) datetime.date(2015, 5, 5)
 datetime.date(2015, 5, 6) datetime.date(2015, 7, 20)
 datetime.date(2015, 9, 21) datetime.date(2015, 9, 22)
 datetime.date(2015, 9, 23) datetime.date(2015, 10, 12)
 datetime.date(2015, 11, 3) datetime.date(2015, 11, 23)
 datetime.date(2015, 12, 23) datetime.date(2016, 1, 1)
 datetime.date(2016, 1, 11) datetime.date(2016, 2, 11)
 datetime.date(2016, 3, 20) datetime.date(2016, 3, 21)
 datetime.date(2016, 4, 29) datetime.date(2016, 5, 3)
 datetime.date(2016, 5, 4) datetime.date(2016, 5, 5)
 datetime.date(2016, 7, 18) datetime.date(2016, 8, 11)
 datetime.date(2016, 9, 19) datetime.date(2016, 9, 22)
 datetime.date(2016, 10, 10) datetime.date(2016, 11, 3)
 datetime.date(2016, 11, 23) datetime.date(2016, 12, 23)
 datetime.date(2017, 1, 1) datetime.date(2017, 1, 2)
 datetime.date(

In [44]:
def get_workdays_jp(start_datetime, end_datetime, return_as="dt", end_include=False):
    """
    営業日を取得
    
    start_datetime: datetime.datetime
        開始時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    end_datetime: datetime.datetime
        終了時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    return_as: str, defalt: 'dt'
        返り値の形式
        - 'dt':pd.DatetimeIndex
        - 'date': datetime.date array
        - 'datetime': datetime.datetime array 
    end_include: bool
        最終日も含めて出力するか
    """
    # 返り値の形式の指定
    return_as_set = {"dt", "datetime", "date"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
    
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)
    
    # 期間中のholidayを取得
    holidays_in_span_index = (start_datetime<=holidays_datetimeindex)&(holidays_datetimeindex<end_datetime)  # DatetimeIndexを使うことに注意
    holidays_in_span_array = holidays_date_array[holidays_in_span_index]  # ndarrayを使う

    # 期間中のdatetimeのarrayを取得
    if end_include:
        days_datetimeindex = pd.date_range(start=start_datetime, end=end_datetime, freq="D")  # 最終日も含める
    else:
        days_datetimeindex = pd.date_range(start=start_datetime, end=end_datetime-datetime.timedelta(days=1), freq="D")  # 最終日は含めない
    
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得
    holiday_bool_array = np.in1d(days_datetimeindex.date, holidays_in_span_array)  # 休日であるかのブール(pd.DatetimeIndex.isin)でもいい
    sun_or_satur_bool_array = (days_datetimeindex.weekday==5) | (days_datetimeindex.weekday==6)  # 土曜日or日曜日
    
    workdays_bool_array = (~holiday_bool_array)&(~sun_or_satur_bool_array)  # 休日でなく土日でない
    
    workdays_datetimeindex = days_datetimeindex[workdays_bool_array].copy()
    if return_as=="dt":
        return workdays_datetimeindex
    elif return_as=="date":
        return workdays_datetimeindex.date
    elif return_as=="datetime":
        workdays_array = workdays_datetimeindex.to_pydatetime()
        return workdays_array

In [45]:
#start_datetime = datetime.datetime(2019, 1, 1, 0, 0, 0)
#end_datetime = datetime.datetime(2019, 12, 31, 0, 0, 0)

jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020, 1, 1, 0, 0, 0))
end_datetime = jst_timezone.localize(datetime.datetime(2020, 12, 31, 0, 0, 0))

workdays = get_workdays_jp(start_datetime, end_datetime, return_as="date", end_include=True)

In [46]:
workdays

array([datetime.date(2020, 1, 2), datetime.date(2020, 1, 3),
       datetime.date(2020, 1, 6), datetime.date(2020, 1, 7),
       datetime.date(2020, 1, 8), datetime.date(2020, 1, 9),
       datetime.date(2020, 1, 10), datetime.date(2020, 1, 14),
       datetime.date(2020, 1, 15), datetime.date(2020, 1, 16),
       datetime.date(2020, 1, 17), datetime.date(2020, 1, 20),
       datetime.date(2020, 1, 21), datetime.date(2020, 1, 22),
       datetime.date(2020, 1, 23), datetime.date(2020, 1, 24),
       datetime.date(2020, 1, 27), datetime.date(2020, 1, 28),
       datetime.date(2020, 1, 29), datetime.date(2020, 1, 30),
       datetime.date(2020, 1, 31), datetime.date(2020, 2, 3),
       datetime.date(2020, 2, 4), datetime.date(2020, 2, 5),
       datetime.date(2020, 2, 6), datetime.date(2020, 2, 7),
       datetime.date(2020, 2, 10), datetime.date(2020, 2, 12),
       datetime.date(2020, 2, 13), datetime.date(2020, 2, 14),
       datetime.date(2020, 2, 17), datetime.date(2020, 2, 18),
   

### 営業日であるかどうか 

In [111]:
def check_workday_jp(select_datetime):
    """
    与えられたdatetime.datetimeが営業日であるかどうかを出力する
    select_datetime: datetime.datetime
        入力するdatetime
    """
    select_datetime_array = np.array([select_datetime.date()])  # データ数が一つのndarray
    # 休日であるかどうか
    is_holiday = np.in1d(select_datetime_array, holidays_date_array).item()  # データ数が一つのため，item
    
    # 土曜日か日曜日であるかどうか
    is_sun_satur = (select_datetime.weekday()==5) or (select_datetime.weekday()==6)
    
    is_workday = (not is_holiday) and (not is_sun_satur)
    
    return is_workday

In [112]:
select_datetime = datetime.datetime(2020, 1, 4, 0, 0, 0)

#jst_timezone = timezone("Asia/Tokyo")
#select_datetime = jst_timezone.localize(datetime.datetime(2020, 1, 4, 0, 0, 0))

check_workday_jp(select_datetime)

False

### 非営業日を取得

同様に，すでに取得したholidayと照らし合わせる．

In [52]:
def get_not_workdays_jp(start_datetime, end_datetime, return_as="dt", end_include=False):
    """
    非営業日を取得(土日or祝日)
    
    start_datetime: datetime.datetime
        開始時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    end_datetime: datetime.datetime
        囚虜時刻のjstのタイムゾーンが指定されたawareなdatetimeか，naiveなdatetime(ローカルなタイムゾーンとして日本を仮定)
    return_as: str, defalt: 'dt'
        返り値の形式
        - 'dt':pd.DatetimeIndex
        - 'date': datetime.date array
        - 'datetime': datetime.datetime array
    end_include: bool
        最終日も含めて出力するか
    """
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)
    
    # 期間中のholidayを取得
    holidays_in_span_index = (start_datetime<=holidays_datetimeindex)&(holidays_datetimeindex<end_datetime)  # DatetimeIndexを使うことに注意
    holidays_in_span_array = holidays_date_array[holidays_in_span_index]  # ndarrayを使う

    # 期間中のdatetimeのarrayを取得
    if end_include:
        days_datetimeindex = pd.date_range(start=start_datetime, end=end_datetime, freq="D")  # 最終日も含める
    else:
        days_datetimeindex = pd.date_range(start=start_datetime, end=end_datetime-datetime.timedelta(days=1), freq="D")  # 最終日は含めない
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得
    holiday_bool_array = np.in1d(days_datetimeindex.date, holidays_in_span_array)  # 休日であるかのブール(pd.DatetimeIndex.isin)でもいい
    sun_or_satur_bool_array = (days_datetimeindex.weekday==5) | (days_datetimeindex.weekday==6)  # 土曜日or日曜日
    
    not_workdays_bool_array = holiday_bool_array | sun_or_satur_bool_array  # 休日あるいは土日
    
    not_workdays_datetimeindex = days_datetimeindex[not_workdays_bool_array].copy()
    if return_as=="dt":
        return not_workdays_datetimeindex
    elif return_as=="date":
        return not_workdays_datetimeindex.date
    elif return_as=="datetime":
        not_workdays_array = not_workdays_datetimeindex.to_pydatetime()        
        return not_workdays_array

In [53]:
#start_datetime = datetime.datetime(2019, 1, 1, 0, 0, 0)
#end_datetime = datetime.datetime(2019, 12, 31, 0, 0, 0)

jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020, 1, 1, 0, 0, 0))
end_datetime = jst_timezone.localize(datetime.datetime(2020, 12, 31, 0, 0, 0))

not_workdays = get_not_workdays_jp(start_datetime, end_datetime, return_as="dt")

In [54]:
not_workdays

DatetimeIndex(['2020-01-01', '2020-01-04', '2020-01-05', '2020-01-11',
               '2020-01-12', '2020-01-13', '2020-01-18', '2020-01-19',
               '2020-01-25', '2020-01-26',
               ...
               '2020-11-28', '2020-11-29', '2020-12-05', '2020-12-06',
               '2020-12-12', '2020-12-13', '2020-12-19', '2020-12-20',
               '2020-12-26', '2020-12-27'],
              dtype='datetime64[ns]', length=120, freq=None)

## 指定した日数後の営業日を取得

ジェネレーターを使うので注意

In [107]:
def get_next_workday_jp(select_datetime, return_as="datetime", days=1):
    """
    select_datetime: datetime.datetime
        指定する日時
    return_as: str, defalt: 'dt'
        返り値の形式
        - 'dt':pd.Timstamp
        - 'date': datetime.date array
        - 'datetime': datetime.datetime array
    """
    # 返り値の形式の指定
    return_as_set = {"dt", "datetime", "date"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
        
    def get_next_workday_jp_gen(select_datetime):
        add_days=1
        while True:
            next_day = select_datetime + datetime.timedelta(days=add_days)
            if check_workday_jp(next_day):
                yield next_day
            add_days += 1
    
    next_workday_gen = get_next_workday_jp_gen(select_datetime)
    for i in range(days):
        next_day = next(next_workday_gen)
    
    if return_as=="datetime":
        return next_day
    elif return_as=="date":
        return next_day.date()
    elif return_as=="dt":
        return pd.Timestamp(next_day)

In [110]:
jst_timezone = timezone("Asia/Tokyo")
select_datetime = jst_timezone.localize(datetime.datetime(2020, 1, 1, 0, 0, 0))

next_workday = get_next_workday_jp(select_datetime, days=6, return_as="dt")
next_workday

Timestamp('2020-01-09 00:00:00+0900', tz='Asia/Tokyo')

## データフレームに関しての処理

### データの用意 

In [20]:
from get_stock_price import StockDatabase
from pathlib import Path

db_path = Path("db/stock_db") / Path("stock.db")
stock_db = StockDatabase(db_path)

In [89]:
jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020,11,1,9,0,0))
end_datetime = jst_timezone.localize(datetime.datetime(2020,12,1,15,0,0))

stock_df = stock_db.search_span(stock_names=["6502"], 
                                start_datetime=start_datetime,
                                end_datetime=end_datetime,
                                freq_str="T",
                                to_tokyo=True,
                               )
stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-02 09:00:00+09:00,2607.0,2620.0,2606.0,2617.0,0.0
2020-11-02 09:01:00+09:00,2619.0,2627.0,2619.0,2627.0,4900.0
2020-11-02 09:02:00+09:00,2634.0,2636.0,2634.0,2634.0,30000.0
2020-11-02 09:03:00+09:00,2636.0,2642.0,2633.0,2636.0,35000.0
2020-11-02 09:04:00+09:00,2638.0,2640.0,2633.0,2638.0,10100.0
...,...,...,...,...,...
2020-12-01 14:55:00+09:00,2950.0,2952.0,2949.0,2951.0,25100.0
2020-12-01 14:56:00+09:00,2950.0,2951.0,2948.0,2949.0,16400.0
2020-12-01 14:57:00+09:00,2950.0,2955.0,2950.0,2954.0,25800.0


### 営業日データを取得

高速化のため，既存のholidayと照らし合わせる方法でとる

In [161]:
def extract_workdays_jp_index(dt_index, return_as="index"):
    """
    pd.DatetimeIndexから，営業日のデータのものを抽出
    dt_index: pd.DatetimeIndex
        入力するDatetimeIndex
    return_as: str
        出力データの形式
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
 
    # 返り値の形式の指定
    return_as_set = {"index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
        
    # すでにtimestampでソートさてている前提
    start_datetime = dt_index[0].to_pydatetime()
    end_datetime = dt_index[-1].to_pydatetime()
    
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)
    
    # 期間内のholidayを取得
    holidays_in_span_index = ((start_datetime-datetime.timedelta(days=1))<holidays_datetimeindex)&\
    (holidays_datetimeindex<=end_datetime)  # DatetimeIndexを使うことに注意, 当日を含めるため，startから1を引いている．
    holidays_in_span_array = holidays_date_array[holidays_in_span_index]  # ndarrayを使う
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得    
    holiday_bool_array = np.in1d(dt_index.date, holidays_in_span_array)  # 休日
    sun_or_satur_bool_array = (dt_index.weekday==5) | (dt_index.weekday==6)  # 土曜日or日曜日
    
    workdays_bool_array = (~holiday_bool_array)&(~sun_or_satur_bool_array)  # 休日でなく土日でない
    if return_as=="bool":  # boolで返す場合
        return workdays_bool_array
    
    elif return_as=="index":  # indexで返す場合
        workdays_df_indice = dt_index[workdays_bool_array]
        return workdays_df_indice

In [186]:
def extract_workdays_jp(df, return_as="df"):
    """
    データフレームから，営業日のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
    
    workdays_bool_array = extract_workdays_jp_index(df.index, return_as="bool")
    if return_as=="bool":
        return workdays_bool_array
    
    workdays_df_indice = df.index[workdays_bool_array]
    if return_as=="index":
        return workdays_df_indice

    out_df = df.loc[workdays_df_indice].copy()
    return out_df

In [187]:
extracted_stock_df = extract_workdays_jp(stock_df, return_as="df")
extracted_stock_df.at_time(datetime.time(9,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-02 09:00:00+09:00,2607.0,2620.0,2606.0,2617.0,0.0
2020-11-04 09:00:00+09:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00+09:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00+09:00,2780.0,2782.0,2770.0,2777.0,142700.0
2020-11-11 09:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-12 09:00:00+09:00,2788.0,2800.0,2781.0,2789.0,108400.0
2020-11-13 09:00:00+09:00,2750.0,2754.0,2741.0,2750.0,71500.0


### 日中データを取得

おそらく使わない，以下の営業日＋日中内で使う．

In [172]:
def extract_intraday_jp_index(dt_index, return_as="index"):
    """
    pd.DatetimeIndexから，日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    dt_index: pd.DatetimeIndex
        入力するDatetimeIndex
    return_as: str
        出力データの形式
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
  
    bool_array = np.array([False]*len(dt_index))
    
    # 午前をTrueに
    am_indice = dt_index.indexer_between_time(start_time=datetime.time(9,0), end_time=datetime.time(11,30), include_end=False)
    bool_array[am_indice] = True
    
    # 午後をTrueに
    pm_indice = dt_index.indexer_between_time(start_time=datetime.time(12,30), end_time=datetime.time(15,0), include_end=False)
    bool_array[pm_indice] = True
    
    if return_as=="bool":
        return bool_array

    elif return_as=="index":
        intraday_indice = dt_index[bool_array]
        return intraday_indice

In [184]:
def extract_intraday_jp(df, return_as="df"):
    """
    データフレームから，日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
  
    intraday_bool_array = extract_intraday_jp_index(df.index, return_as="bool")
    if return_as=="bool":
        return intraday_bool_array
    
    intraday_indice = df.index[intraday_bool_array]
    if return_as=="index":
        return intraday_indice
    
    out_df = df.loc[intraday_indice].copy()
    return out_df

In [185]:
extracted_df = extract_intraday_jp(stock_df, return_as="df")
extracted_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-02 09:00:00+09:00,2607.0,2620.0,2606.0,2617.0,0.0
2020-11-02 09:01:00+09:00,2619.0,2627.0,2619.0,2627.0,4900.0
2020-11-02 09:02:00+09:00,2634.0,2636.0,2634.0,2634.0,30000.0
2020-11-02 09:03:00+09:00,2636.0,2642.0,2633.0,2636.0,35000.0
2020-11-02 09:04:00+09:00,2638.0,2640.0,2633.0,2638.0,10100.0
...,...,...,...,...,...
2020-12-01 14:55:00+09:00,2950.0,2952.0,2949.0,2951.0,25100.0
2020-12-01 14:56:00+09:00,2950.0,2951.0,2948.0,2949.0,16400.0
2020-12-01 14:57:00+09:00,2950.0,2955.0,2950.0,2954.0,25800.0


### 営業日＋日中データを取得 

In [175]:
def extract_workdays_intraday_jp_index(dt_index, return_as="index"):
    """
    pd.DatetimeIndexから，営業日+日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    dt_index: pd.DatetimeIndex
        入力するDatetimeIndex
    return_as: str
        出力データの形式
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """

    # 返り値の形式の指定
    return_as_set = {"index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
        
    workday_bool_array = extract_workdays_jp_index(dt_index, return_as="bool")
    intraday_bool_array = extract_intraday_jp_index(dt_index, return_as="bool")
    
    workday_intraday_bool_array = workday_bool_array & intraday_bool_array
    if return_as=="bool":
        return workday_intraday_bool_array
    elif return_as=="index":
        workday_intraday_indice = dt_index[workday_intraday_bool_array]
        return workday_intraday_indice

In [180]:
jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020,11,1,9,0,0))
end_datetime = jst_timezone.localize(datetime.datetime(2020,12,1,15,0,0))

datetime_index = pd.date_range(start_datetime, end_datetime, freq="5T")
datetime_index[datetime_index.indexer_at_time(datetime.time(9,0))]

intraday_workday_index = extract_workdays_intraday_jp_index(datetime_index, return_as="index")
intraday_workday_index[intraday_workday_index.indexer_at_time(datetime.time(9,0))]

DatetimeIndex(['2020-11-02 09:00:00+09:00', '2020-11-04 09:00:00+09:00',
               '2020-11-05 09:00:00+09:00', '2020-11-06 09:00:00+09:00',
               '2020-11-09 09:00:00+09:00', '2020-11-10 09:00:00+09:00',
               '2020-11-11 09:00:00+09:00', '2020-11-12 09:00:00+09:00',
               '2020-11-13 09:00:00+09:00', '2020-11-16 09:00:00+09:00',
               '2020-11-17 09:00:00+09:00', '2020-11-18 09:00:00+09:00',
               '2020-11-19 09:00:00+09:00', '2020-11-20 09:00:00+09:00',
               '2020-11-24 09:00:00+09:00', '2020-11-25 09:00:00+09:00',
               '2020-11-26 09:00:00+09:00', '2020-11-27 09:00:00+09:00',
               '2020-11-30 09:00:00+09:00', '2020-12-01 09:00:00+09:00'],
              dtype='datetime64[ns, Asia/Tokyo]', freq=None)

In [181]:
def extract_workdays_intraday_jp(df, return_as="df"):
    """
    データフレームから，営業日+日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
       
    workday_intraday_bool_array = extract_workdays_intraday_jp_index(df.index, return_as="bool")
    
    if return_as=="bool":
        return workday_intraday_bool_array
    
    workday_intraday_indice = df.index[workday_intraday_bool_array]
    
    if return_as=="index":
        return workday_intraday_indice
    
    out_df = df.loc[workday_intraday_indice].copy()
    return out_df

In [183]:
extracted_df = extract_workdays_intraday_jp(stock_df, return_as="df")
extracted_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-02 09:00:00+09:00,2607.0,2620.0,2606.0,2617.0,0.0
2020-11-02 09:01:00+09:00,2619.0,2627.0,2619.0,2627.0,4900.0
2020-11-02 09:02:00+09:00,2634.0,2636.0,2634.0,2634.0,30000.0
2020-11-02 09:03:00+09:00,2636.0,2642.0,2633.0,2636.0,35000.0
2020-11-02 09:04:00+09:00,2638.0,2640.0,2633.0,2638.0,10100.0
...,...,...,...,...,...
2020-12-01 14:55:00+09:00,2950.0,2952.0,2949.0,2951.0,25100.0
2020-12-01 14:56:00+09:00,2950.0,2951.0,2948.0,2949.0,16400.0
2020-12-01 14:57:00+09:00,2950.0,2955.0,2950.0,2954.0,25800.0
